<a href="https://colab.research.google.com/github/fikrinotes/LSTM-IDS/blob/main/Final_Project_IDS_with_Autoencoder_and_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

dataset_path = kagglehub.dataset_download('chethuhn/network-intrusion-dataset')
model_path = kagglehub.dataset_download('fikrimulyanasetiawan/rnn-model')
encoder_path = kagglehub.dataset_download('fikrimulyanasetiawan/encoder')

print('Data source import complete.')


In [ ]:
print(dataset_path)

# Intrusion Detection System

## Import Library

In [ ]:
# Import library yang diperlukan
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
import tensorflow as tf
from tensorflow.keras import layers, Model
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns


## Preprocessing Data

In [ ]:
# Fungsi untuk membaca dan preprocessing setiap file
def read_and_clean_file(file_path):
    print(f"Membaca file: {file_path}")
    df = pd.read_csv(file_path, low_memory=False, sep=",")

    # Bersihkan nama kolom dari whitespace
    df.columns = df.columns.str.strip()

    # Hapus kolom yang tidak diperlukan
    redundant = ['Flow ID', 'Source IP', 'Source Port', 'Destination IP',
                 'Destination Port', 'Protocol', 'Timestamp']
    df = df.drop(redundant, axis=1, errors='ignore')

    # drop baris yang tidak punya label
    df.dropna(subset = ['Label'], inplace=True)

    # Handling missing values dan infinite values
    df = df.replace([np.inf, -np.inf], np.nan)

    return df


# Baca semua file CSV dari folder
folder_path = '/kaggle/input/network-intrusion-dataset'
data1 = dataset_path + "/Monday-WorkingHours.pcap_ISCX.csv"
data2 = dataset_path + "/Tuesday-WorkingHours.pcap_ISCX.csv"
data3 = dataset_path + "/Wednesday-workingHours.pcap_ISCX.csv"
data4 = dataset_path + "/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv"
data5 = dataset_path + "/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv"
data6 = dataset_path + "/Friday-WorkingHours-Morning.pcap_ISCX.csv"
data7 = dataset_path + "/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv"
data8 = dataset_path + "/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"


# Baca semua file CSV dari folder
all_files = [data2, data3, data4, data5, data6, data7, data8]

# Membaca file dan mengkonversi data file menjadi dataframe
dataframes = []
for file in all_files:
    # file_path = os.path.join(folder_path, file)
    df = read_and_clean_file(file)
    dataframes.append(df)

In [ ]:
# Menggabungkan semua dataframe
print("Menggabungkan semua file...")
df = pd.concat(dataframes, ignore_index=True)
try:
    print("Semua file dataset berhasil digabungkan!")
except:
    print("Error! file dataset tidak berhasil digabungkan")

In [ ]:
dataframe = df.copy(deep=True)

In [ ]:
dataframe['Label'] = dataframe['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)

In [ ]:
df.columns

In [ ]:
corr_matrix = dataframe.corr(method='pearson')

In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)

In [ ]:
corr_matrix

In [ ]:
# Korelasi setiap fitur dengan label
target_corr = corr_matrix['Label'].abs().sort_values(ascending=False)

# Pilih fitur dengan korelasi > threshold (misal: top 20%)
threshold = 0.05  # Ambang minimal korelasi
high_corr_features = target_corr[target_corr > threshold].index.tolist()

# Hapus 'Label' jika termasuk
high_corr_features = [f for f in high_corr_features if f != 'Label']

print(f"Selected features: {len(high_corr_features)}")
print(high_corr_features)

In [ ]:
# Buat matriks korelasi hanya untuk fitur terpilih
high_corr_df = dataframe[high_corr_features]
feature_corr = high_corr_df.corr().abs()

# Identifikasi pasangan fitur dengan korelasi tinggi
to_drop = set()
for i in range(len(feature_corr.columns)):
    for j in range(i):
        if feature_corr.iloc[i, j] > 0.8:  # Threshold multikolinearitas
            col_i = feature_corr.columns[i]
            col_j = feature_corr.columns[j]

            # Bandingkan korelasi dengan target, hapus yang lebih rendah
            if target_corr[col_i] > target_corr[col_j]:
                to_drop.add(col_j)
            else:
                to_drop.add(col_i)

# Final feature selection
final_features = [f for f in high_corr_features if f not in to_drop]

print(f"\nFinal features after multicollinearity check: {len(final_features)}")
print(final_features)

In [ ]:
df.describe()

In [ ]:
df['Label']

### Pembersihan data duplikat

In [ ]:
# ganti nama kolom dengan cara hapus whitespaces
col_names = {col: col.strip() for col in df.columns}
df.rename(columns = col_names, inplace = True)

# informasi data duplikat
dups = df[df.duplicated()]
print(f'Number of duplicates: {len(dups)}')

In [ ]:
df.shape

In [ ]:
# Hapus data duplikat
df.drop_duplicates(inplace = True)
df.shape

## Informasi Umum Dataset

In [ ]:
# Menampilkan informasi dataset
print("\nInformasi Dataset:")
print(f"\nJumlah total data: {len(df)}")
print(f"Jumlah fitur : {len(df.columns)}")
print("\nDistribusi Label sebelum preprocessing:")

# tabel distribusi label
def create_distribution_table(df):
    label_dist = pd.DataFrame(df['Label'].value_counts())
    label_dist['percentage'] = df['Label'].value_counts()/len(df)
    return label_dist

create_distribution_table(df)

In [ ]:
df["Label"] = df["Label"].where(df["Label"] == "BENIGN", "ANOMALY")

In [ ]:
df["Label"].unique()

In [ ]:
df

In [ ]:
# Menampilkan informasi dataset
print("\nInformasi Dataset:")
print(f"\nJumlah total data: {len(df)}")
print(f"Jumlah fitur : {len(df.columns)}")
print("\nDistribusi Label setelah preprocessing:")

create_distribution_table(df)

## Pemisahan Data Fitur (X) dan Ouput (y)

In [ ]:
df[final_features].info()

In [ ]:
numerical_columns = df.select_dtypes(include=[np.number]).columns
X = df[final_features]
y = df["Label"]

In [ ]:
X

## Training-Test Split

In [ ]:
tss = TimeSeriesSplit(n_splits=7)
print(tss)

In [ ]:
#X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
train_index, test_index = [], []
for i, (train_interval, test_interval) in enumerate(tss.split(X)):
    print(f"fold {i}:")
    print(f"  Train: index : from {train_interval.min()} up to {train_interval.max()}")
    print(f"  Test:  index=from {test_interval.min()} up to {test_interval.max()}")
    print(f"  Jumlah kelas pada training set : {y.iloc[train_interval].nunique()}")
    print(f"  Jumlah kelas pada testing set : {y.iloc[test_interval].nunique()}")
    train_index, test_index = train_interval, test_interval


In [ ]:
# Split dataset dengan stratifikasi
X_train, X_test = X.iloc[train_index], X.iloc[test_index]
y_train, y_test  = y.iloc[train_index], y.iloc[test_index]

In [ ]:
y_test.value_counts()

In [ ]:
y_train.value_counts()

## Transformasi Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis

### Penanganan Missing Values, Normalisasi Data dan Label Encoding

In [ ]:
# imputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean', copy=False)
print("fitting imputer...")
imputer.fit(X_train)
print("selesai!")

# scaler
scaler = StandardScaler(copy=False)
print("\nfitting scaler...")
scaler.fit(X_train)
print("selesai!")

# label encoder (le)
le = LabelEncoder()
print("\nfitting label encoder...")
le.fit(y_train.astype(str))
print("selesai!")

In [ ]:
df["Label"].value_counts()

In [ ]:
def transform_data(X, y, scaler, imputer, le):
    # Handling missing values untuk dataset training
    print("\nMenangani missing values...")
    X = imputer.transform(X)
    print("selesai!")

    # Normalisasi Data
    print("\nMelakukan normalisasi data...")
    X = scaler.transform(X)
    print("selesai!")

    # One-hot encoding untuk target (karena multi-kelas)
    num_classes = len(le.classes_)
    print("\nMelakukan one-hot encoding...")
    y = le.transform(y)
    y = tf.keras.utils.to_categorical(y, num_classes)
    print("selesai!")

    return X, y

In [ ]:
y_train

In [ ]:
num_classes = len(le.classes_)

In [ ]:
num_classes

In [ ]:
## Transformasi Data Training
X_train, y_train = transform_data(X_train, y_train, scaler, imputer, le)

In [ ]:
le.classes_

In [ ]:
y_train

In [ ]:
for i, label in enumerate(le.classes_):
    print(f"i : {i} , label : {label}")

In [ ]:
# Menampilkan informasi kelas
print("\nKelas yang terdeteksi:")
for i, label in enumerate(le.classes_):
    count = (df["Label"] == i).sum()
    print(f"{label}: {count} samples (encoded as {i})")


### Penggunaan TPU

In [ ]:
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="local")

# # instantiate a distribution strategy
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.TPUStrategy(tpu)


## Konstruksi Model Autoencoder

In [ ]:
y_test

In [ ]:
# Transformmasi data testing
X_test, y_test = transform_data(X_test, y_test, scaler, imputer, le)

In [ ]:
# # Membuat Autoencoder
# input_dim = X_train.shape[1]
# encoding_dim = 40  # Meningkatkan dimensi encoding karena data lebih kompleks. saran : 64

# # Encoder
# input_layer = layers.Input(shape=(input_dim,))

# encoded = layers.Dense(256, activation='relu')(input_layer)
# encoded = layers.BatchNormalization()(encoded)
# encoded = layers.Dropout(0.2)(encoded)

# encoded = layers.Dense(128, activation='relu')(encoded)
# encoded = layers.BatchNormalization()(encoded)
# encoded = layers.Dropout(0.2)(encoded)

# encoded = layers.Dense(64, activation='relu')(encoded)
# encoded = layers.BatchNormalization()(encoded)
# encoded = layers.Dropout(0.2)(encoded)

# # Bottleneck
# encoded = layers.Dense(encoding_dim, activation='linear')(encoded)

# # Decoder
# decoded = layers.Dense(64, activation='leaky_relu')(encoded) # coba leaky_relu
# decoded = layers.BatchNormalization()(decoded)
# decoded = layers.Dropout(0.2)(decoded)

# decoded = layers.Dense(128, activation='leaky_relu')(decoded)
# decoded = layers.BatchNormalization()(decoded)
# decoded = layers.Dropout(0.2)(decoded)

# decoded = layers.Dense(256, activation='leaky_relu')(decoded)
# decoded = layers.BatchNormalization()(decoded)
# decoded = layers.Dropout(0.2)(decoded)

# decoded = layers.Dense(input_dim, activation='linear')(decoded)

# # Model Autoencoder
# autoencoder = Model(input_layer, decoded)
# encoder = Model(input_layer, encoded)

# # Compile dengan learning rate yang sesuai
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
# autoencoder.compile(optimizer=optimizer, loss='mse')

In [ ]:

# # Training Autoencoder dengan early stopping
# early_stopping = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss',
#     patience=5,
#     restore_best_weights=True
# )

# print("\nTraining Autoencoder...")
# # history_autoencoder = autoencoder.fit(X_train, X_train,
# #                                     epochs=50,
# #                                     batch_size=512,  # Meningkatkan batch size
# #                                     shuffle=True,
# #                                     validation_split=0.2,
# #                                     callbacks=[early_stopping])
# history_autoencoder = autoencoder.fit(X_train, X_train,
#                                     epochs=200,
#                                     batch_size=256,
#                                     validation_data=(X_test, X_test))


In [ ]:
# # Mendapatkan encoded features
# X_train_encoded = encoder.predict(X_train)

In [ ]:
# plt.plot(history_autoencoder.history['loss'], label='Training Loss')
# plt.plot(history_autoencoder.history['val_loss'], label='Validation Loss')
# plt.title('Autoencoder Training History')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.savefig("ae_plot_history")

In [ ]:
# from tensorflow.keras.utils import plot_model
# plot_model(autoencoder, to_file='model.png')

In [ ]:
# autoencoder.summary()

## Konstruksi Model LSTM

In [ ]:
y_test.shape

In [ ]:
input_dim = X_train.shape[1]

In [ ]:
# Membuat model LSTM untuk multi-kelas dengan data dari Autoencoder
rnn_model = tf.keras.Sequential([
    layers.Input(shape=(input_dim,)), # ini encoding_dim (sebenarnya)
    layers.Reshape((input_dim, 1)),
    layers.LSTM(256, return_sequences=True),
    layers.Dropout(0.3),
    layers.LSTM(128),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Output layer untuk multi-kelas, gunakan num_classes utk jumlah neuron
]) # jika cuma 2 kelas (seperti deteksi anomali, cukup 1 neuron pada layer output)

# Compile RNN
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
rnn_model.compile(optimizer=optimizer,
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

# Training RNN dengan early stopping
early_stopping_rnn = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

print("\nTraining RNN...")
# history_rnn = rnn_model.fit(X_train, y_train,
#                            epochs=100,
#                            batch_size=512,
#                            validation_data=(X_test, y_test),
#                            callbacks=[early_stopping_rnn])
history_rnn = rnn_model.fit(X_train, y_train,
                           epochs=100,
                           batch_size=256,
                           validation_data=(X_test, y_test)
                           )


In [ ]:
plt.plot(history_rnn.history['accuracy'], label='Training Accuracy')
plt.plot(history_rnn.history['val_accuracy'], label='Validation Accuracy')
plt.title('LSTM Training History')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig("lstm_training_history")

In [ ]:
# Membuat model RNN untuk multi-kelas tanpa data dari Autoencoder
# encoding_dim = 78
# rnn_model = tf.keras.Sequential([
#     layers.Input(shape=(encoding_dim,)),
#     layers.Reshape((encoding_dim, 1)),
#     layers.LSTM(256, return_sequences=True),
#     layers.Dropout(0.4),
#     layers.LSTM(128),
#     layers.Dropout(0.4),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(num_classes, activation='softmax')  # Output layer untuk multi-kelas
# ])

# # Compile RNN
# rnn_model.compile(optimizer='adam',
#                  loss='categorical_crossentropy',
#                  metrics=['accuracy'])

# # Training RNN dengan early stopping
# early_stopping_rnn = tf.keras.callbacks.EarlyStopping(
#     monitor='val_accuracy',
#     patience=5,
#     restore_best_weights=True
# )

# print("\nTraining RNN...")
# history_rnn = rnn_model.fit(X_train, y_train,
#                            epochs=50,
#                            batch_size=512,
#                            validation_split=0.2,
#                            callbacks=[early_stopping_rnn])


In [ ]:
rnn_model.summary()

## Evaluasi Model pada Data Test

In [ ]:
X_test, y_test = transform_data(X_test, y_test, scaler, imputer, le)
X_test_encoded = encoder.predict(X_test)

In [ ]:
Evaluasi model
y_pred = rnn_model.predict(X_test_encoded)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Tampilkan hasil evaluasi
print("\nClassification Report:")
print(classification_report(y_test_classes, y_pred_classes, target_names=le.classes_))

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history_autoencoder.history['loss'], label='Training Loss')
plt.plot(history_autoencoder.history['val_loss'], label='Validation Loss')
plt.title('Autoencoder Training History')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_rnn.history['accuracy'], label='Training Accuracy')
plt.plot(history_rnn.history['val_accuracy'], label='Validation Accuracy')
plt.title('RNN Training History')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
plt.plot(history_autoencoder.history['loss'], label='Training Loss')
plt.plot(history_autoencoder.history['val_loss'], label='Validation Loss')
plt.title('Autoencoder Training History')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig("ae_plot_history")

In [ ]:
plt.plot(history_rnn.history['accuracy'], label='Training Accuracy')
plt.plot(history_rnn.history['val_accuracy'], label='Validation Accuracy')
plt.title('LSTM Training History')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig("lstm_training_history")

In [ ]:
# # Plot confusion matrix
# plt.figure(figsize=(12, 10))
# cm = confusion_matrix(y_test_classes, y_pred_classes)
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
#             xticklabels=le.classes_,
#             yticklabels=le.classes_)
# plt.title('Confusion Matrix')
# plt.ylabel('True Label')
# plt.xlabel('Predicted Label')
# plt.xticks(rotation=45)
# plt.yticks(rotation=45)
# plt.tight_layout()
# plt.show()
# plt.savefig("confusion_matrix")


In [ ]:
# Simpan model
print("\nMenyimpan model...")
autoencoder.save('autoencoder_model.h5')
encoder.save('encoder_model.h5')
rnn_model.save('rnn_model.h5')

In [ ]:
# Simpan label encoder
import joblib
joblib.dump(le, 'label_encoder.joblib')
joblib.dump(scaler, 'scaler.joblib')

---